In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

cnt = 0
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        cnt+=1
print(f" Total number of files are : {cnt}")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 Total number of files are : 58142


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import cv2 as cv
import os
from tensorflow import keras
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

In [4]:
train_dir = r'/kaggle/input/animals-detection-images-dataset/train/'
test_dir = r'/kaggle/input/animals-detection-images-dataset/test/'

labels = ['Butterfly',
       'Deer', 'Duck', 'Eagle','Fish',
       'Penguin']

label_len = len(labels)
print(label_len)

6


In [5]:
X_train = []
Y_train = []
for label in labels:
    folderpath = os.path.join(train_dir, label)
    
    for file in os.listdir(folderpath):
        img_path = os.path.join(folderpath, file)
        img = cv.imread(img_path)
        
        if img is not None:
            img = cv.resize(img, (224, 224))
            X_train.append(np.array(img))
            Y_train.append(labels.index(label))

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [6]:
print(len(X_train))

4675


In [7]:
X_valid = []
Y_valid = []
X_valid_path = []

for label in labels:
    folderpath = os.path.join(test_dir, label)
    
    for file in os.listdir(folderpath):
        img_path = os.path.join(folderpath, file)
        img = cv.imread(img_path)
        
        if img is not None:
            img = cv.resize(img, (224, 224))
            X_valid.append(np.array(img))
            X_valid_path.append(img_path)
            Y_valid.append(labels.index(label))
            
X_valid = np.array(X_valid)
Y_valid = np.array(Y_valid)

In [8]:
base_model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [9]:
base_model.trainable = False

In [10]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(1024,activation='relu')(x)
x = Dense(512, activation='relu')(x)
pred = Dense(len(labels),activation='softmax')(x)

In [11]:
model = Model(inputs=base_model.input,outputs = pred)

In [12]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
Y_train_oh = to_categorical(Y_train)
Y_valid_oh = to_categorical(Y_valid)

In [14]:
history = model.fit(X_train,Y_train_oh,epochs=10,validation_data=(X_valid,Y_valid_oh))

Epoch 1/10
  2/147 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - accuracy: 0.1641 - loss: 2.2569 

I0000 00:00:1721578084.768329    1016 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1721578084.817966    1016 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.8081 - loss: 0.6922

W0000 00:00:1721578100.466251    1018 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1721578104.960747    1018 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


147/147 ━━━━━━━━━━━━━━━━━━━━ 44s 192ms/step - accuracy: 0.8087 - loss: 0.6898 - val_accuracy: 0.9225 - val_loss: 0.2415
Epoch 2/10
  2/147 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - accuracy: 0.9062 - loss: 0.2424

W0000 00:00:1721578112.948108    1016 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


147/147 ━━━━━━━━━━━━━━━━━━━━ 14s 95ms/step - accuracy: 0.9641 - loss: 0.1148 - val_accuracy: 0.9311 - val_loss: 0.2112
Epoch 3/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - accuracy: 0.9816 - loss: 0.0567 - val_accuracy: 0.9698 - val_loss: 0.0952
Epoch 4/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 14s 98ms/step - accuracy: 0.9857 - loss: 0.0385 - val_accuracy: 0.9729 - val_loss: 0.1088
Epoch 5/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 15s 99ms/step - accuracy: 0.9860 - loss: 0.0369 - val_accuracy: 0.9760 - val_loss: 0.0862
Epoch 6/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 14s 99ms/step - accuracy: 0.9849 - loss: 0.0389 - val_accuracy: 0.9589 - val_loss: 0.1755
Epoch 7/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 14s 98ms/step - accuracy: 0.9899 - loss: 0.0316 - val_accuracy: 0.9063 - val_loss: 0.3814
Epoch 8/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 14s 97ms/step - accuracy: 0.9955 - loss: 0.0144 - val_accuracy: 0.9713 - val_loss: 0.1462
Epoch 9/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 20s 97ms/step - accuracy: 0.9992 - loss: 0.0037 - val_accurac

In [15]:
model.predict(X_valid)

 3/41 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step

W0000 00:00:1721578252.356185    1019 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


41/41 ━━━━━━━━━━━━━━━━━━━━ 11s 172ms/step


W0000 00:00:1721578259.226156    1016 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


array([[1.00000000e+00, 1.70270716e-14, 1.71306942e-14, 3.66456565e-15,
        1.61456661e-08, 3.01648317e-15],
       [1.00000000e+00, 7.24959322e-23, 2.85357722e-21, 1.49618729e-22,
        5.05607095e-16, 1.09308997e-22],
       [1.00000000e+00, 1.18817975e-26, 9.12965607e-24, 5.36071738e-27,
        2.85506861e-21, 1.20794103e-26],
       ...,
       [1.20976211e-19, 7.97636048e-20, 1.52857865e-17, 1.18251435e-18,
        4.02327812e-16, 1.00000000e+00],
       [7.48777600e-14, 3.06221539e-19, 6.00224064e-16, 6.54653071e-18,
        3.00985945e-17, 1.00000000e+00],
       [7.72761741e-21, 8.22555805e-23, 1.14128009e-19, 1.83208338e-22,
        2.94549454e-22, 1.00000000e+00]], dtype=float32)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open('/kaggle/working/model.tflite','wb').write(tflite_model)

In [2]:
from tensorflow import keras
import tensorflow as tf

2024-07-21 16:18:55.700508: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 16:18:55.700592: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 16:18:55.702113: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
keras_file = "/kaggle/working/model.h5"
model = keras.models.load_model(keras_file)

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)